In [1]:
from selenium import webdriver
from time import sleep
import time
from selenium.webdriver.common.keys import Keys
from tqdm import tqdm
import csv

PATH = "msedgedriver.exe"

categories = ['DIENTHOAI', 'MAYTINHBANG', 'VOBAOOPLUNG','MIENGDAN','PINSAC' , 'CAPSAC', 'GAYCHUPHINH', 'SIM',
             'LAPTOP', 'MAYBAN','LINHKIENMAYTINH','CHUOTBANPHIM','THIETBIMANG','OCUNG','PHUKIEN','AUDIO']

DIENTHOAI = 'https://shopee.vn/%C4%90i%E1%BB%87n-tho%E1%BA%A1i-cat.84.1979'
MAYTINHBANG = 'https://shopee.vn/M%C3%A1y-t%C3%ADnh-b%E1%BA%A3ng-cat.84.14026'
VOBAOOPLUNG = 'https://shopee.vn/V%E1%BB%8F-Bao-%E1%BB%90p-l%C6%B0ng-cat.84.1985'
MIENGDAN = 'https://shopee.vn/Mi%E1%BA%BFng-d%C3%A1n-m%C3%A0n-h%C3%ACnh-cat.84.2802'
PINSAC = 'https://shopee.vn/Pin-s%E1%BA%A1c-d%E1%BB%B1-ph%C3%B2ng-cat.84.2817'
CAPSAC = 'https://shopee.vn/Pin-C%C3%A1p-B%E1%BB%99-s%E1%BA%A1c-cat.84.1981'
GAYCHUPHINH = 'https://shopee.vn/G%E1%BA%ADy-ch%E1%BB%A5p-h%C3%ACnh-cat.84.2403'
SIM = 'https://shopee.vn/Sim-cat.84.2803'

LAPTOP ='https://shopee.vn/Laptop-cat.13030.13065'
MAYBAN = 'https://shopee.vn/M%C3%A1y-T%C3%ADnh-B%C3%A0n-cat.13030.13067'
LINHKIENMAYTINH = 'https://shopee.vn/Linh-Ki%E1%BB%87n-M%C3%A1y-T%C3%ADnh-cat.13030.13069'
CHUOTBANPHIM ='https://shopee.vn/Chu%E1%BB%99t-B%C3%A0n-Ph%C3%ADm-cat.13030.13077'
THIETBIMANG ='https://shopee.vn/Thi%E1%BA%BFt-B%E1%BB%8B-M%E1%BA%A1ng-cat.13030.13079'
OCUNG = 'https://shopee.vn/USB-%E1%BB%94-C%E1%BB%A9ng-cat.13030.13083'
PHUKIEN = 'https://shopee.vn/Ph%E1%BB%A5-Ki%E1%BB%87n-M%C3%A1y-T%C3%ADnh-Kh%C3%A1c-cat.13030.14318'
AUDIO = 'https://shopee.vn/Audio-Video-Accessories-cat.13030.16671'
    
qpage = lambda l,n: l+f'?page={n}'


In [2]:
search_url_dict = dict(zip(categories, 
                           [DIENTHOAI,MAYTINHBANG,VOBAOOPLUNG,MIENGDAN,PINSAC,CAPSAC,GAYCHUPHINH,SIM,
                           LAPTOP,MAYBAN,LINHKIENMAYTINH,CHUOTBANPHIM,THIETBIMANG,OCUNG,PHUKIEN,AUDIO]))

In [4]:
search_url_dict

{'DIENTHOAI': 'https://shopee.vn/%C4%90i%E1%BB%87n-tho%E1%BA%A1i-cat.84.1979',
 'MAYTINHBANG': 'https://shopee.vn/M%C3%A1y-t%C3%ADnh-b%E1%BA%A3ng-cat.84.14026',
 'VOBAOOPLUNG': 'https://shopee.vn/V%E1%BB%8F-Bao-%E1%BB%90p-l%C6%B0ng-cat.84.1985',
 'MIENGDAN': 'https://shopee.vn/Mi%E1%BA%BFng-d%C3%A1n-m%C3%A0n-h%C3%ACnh-cat.84.2802',
 'PINSAC': 'https://shopee.vn/Pin-s%E1%BA%A1c-d%E1%BB%B1-ph%C3%B2ng-cat.84.2817',
 'CAPSAC': 'https://shopee.vn/Pin-C%C3%A1p-B%E1%BB%99-s%E1%BA%A1c-cat.84.1981',
 'GAYCHUPHINH': 'https://shopee.vn/G%E1%BA%ADy-ch%E1%BB%A5p-h%C3%ACnh-cat.84.2403',
 'SIM': 'https://shopee.vn/Sim-cat.84.2803',
 'LAPTOP': 'https://shopee.vn/Laptop-cat.13030.13065',
 'MAYBAN': 'https://shopee.vn/M%C3%A1y-T%C3%ADnh-B%C3%A0n-cat.13030.13067',
 'LINHKIENMAYTINH': 'https://shopee.vn/Linh-Ki%E1%BB%87n-M%C3%A1y-T%C3%ADnh-cat.13030.13069',
 'CHUOTBANPHIM': 'https://shopee.vn/Chu%E1%BB%99t-B%C3%A0n-Ph%C3%ADm-cat.13030.13077',
 'THIETBIMANG': 'https://shopee.vn/Thi%E1%BA%BFt-B%E1%BB%8B-M%E

Crawl dữ liệu từ shopee theo từng category

# Search Page
=> crawl các thông tin sợ lượt về item (tên, giá, lượt bán,...) + url dẫn đến item (1 file riêng)
# Item Page
với mỗi item trong url thì crawl comment, ratings,....

In [5]:
driver = webdriver.Edge(PATH)
driver.maximize_window()


In [14]:
def collect_search_page(driver, search_url_dict, output_filename ="search.csv", pages_per_category=5):
    with open(output_filename, mode='w', encoding='utf-8') as csv_file:
        csv_writer = csv.writer(csv_file, delimiter='\t', quotechar='"', quoting=csv.QUOTE_MINIMAL)
        csv_writer.writerow(['name', 'category','n_sold', 'price', 'shop_address','image_url', 'url'])
        for category, search_url in search_url_dict.items():
            for i in range(pages_per_category):
                driver.get(qpage(search_url,i))
                time.sleep(1)
                total_height = int(driver.execute_script("return document.body.scrollHeight"))

                for i in range(1, total_height, 100):
                    driver.execute_script("window.scrollTo(0, {});".format(i))
                
                items = driver.find_elements_by_class_name("shopee-search-item-result__item")
                
                for i,it in enumerate(items):
                    try:
                        #Get interest attributes
                        name = it.find_element_by_css_selector('._1NoI8_').text
                        n_sold = it.find_element_by_css_selector('._18SLBt').text
                        price = it.find_element_by_css_selector('._1w9jLI._37ge-4._2ZYSiu > span._341bF0').text
                        shop_address = it.find_element_by_css_selector('._3amru2').text   
                        image_url = it.find_element_by_css_selector('img._1T9dHf.V1Fpl5').get_attribute("src")
                        url = it.find_element_by_css_selector('a').get_attribute("href")

                        #reformat some fields
                        print("-".join([name,n_sold,price,shop_address]))
                        n_sold = n_sold.split(" ")[-1] #convert n_sold to numeric
                        if n_sold[-1] == 'k':
                            n_sold = float(n_sold.replace(",",".").strip("k")) * 1000
                        else:
                            n_sold = float(n_sold)
                        price = float(price.replace(".","")) #convert price to numeric

                        csv_writer.writerow([name,category, n_sold, price,shop_address, image_url, url])

                    except: 
                        print('Missing ', i)

                #time.sleep(1)

In [15]:
#collect_search_page(driver, {'DIENTHOAI': search_url_dict['DIENTHOAI']}, pages_per_category=1)
collect_search_page(driver, search_url_dict, pages_per_category=20)

Điện thoại OPPO A37 Fullbox Mới - 2GB/16GB - Bảo hành 12 tháng --Đã bán 7,8k-1.299.000-TP. Hồ Chí Minh
điện thoại Samsung Galaxy S7 chính hãng / full chức năng-Đã bán 6,6k-1.899.000-TP. Hồ Chí Minh
Điện thoại Nokia 1280 Nguyên zin Mới tinh chính hãng-Đã bán 8,8k-189.900-TP. Hồ Chí Minh
ĐIỆN THOẠI NOKIA 1202 MỚI TINH FULL SẠC PIN CHÍNH HÃNG-Đã bán 6,1k-175.000-TP. Hồ Chí Minh
điện thoại Samsung Galaxy S7 Edge 2 sim ram 4G bộ nhớ 32G mới, chơi game nặng mượt-Đã bán 5,4k-2.799.000-TP. Hồ Chí Minh
Điện Thoại Nokia 6300 Siêu Bền _ Đủ Màu _ Chính hãng Nhập Khẩu-Đã bán 7k-290.000-TP. Hồ Chí Minh
điện thoại nokia 110i nghe gọi to rõ kèm pin sạc chính hãng-Đã bán 8,3k-145.000-TP. Hồ Chí Minh
ĐIỆN THOẠI SAMSUNG GALAXY S6 FULLBOX NGUYÊN ZIN CHÍNH HÃNG 100%//SHIP NỘI THÀNH-Đã bán 3,9k-1.818.000-TP. Hồ Chí Minh
Điện Thoại Siêu Dễ Thương Mini BM10-Đã bán 6,1k-169.000-TP. Hồ Chí Minh
Điện thoại Nokia 6300 đầy đủ pin sạc màu bạc-Đã bán 2,7k-289.900-TP. Hồ Chí Minh
Điện thoại Samsung Galaxy A10s (3

In [67]:
import pandas as pd
import numpy as np

In [68]:
df = pd.read_csv("search.csv",encoding='utf-8',sep='\t')

In [69]:
df.category.astype('category').value_counts()

VOBAOOPLUNG        1000
THIETBIMANG        1000
SIM                1000
PINSAC             1000
PHUKIEN            1000
MIENGDAN           1000
LINHKIENMAYTINH    1000
GAYCHUPHINH        1000
CHUOTBANPHIM       1000
CAPSAC             1000
AUDIO              1000
LAPTOP              999
MAYBAN              998
DIENTHOAI           997
OCUNG               950
MAYTINHBANG         704
Name: category, dtype: int64